# Validation

This notebook depends on data generated by 'MG_04-04-2022_Running_Moscot'

In [1]:
import scanpy as sc
import numpy as np
import networkx as nx
import anndata
import pandas as pd
import ot
import sklearn.metrics

In [2]:
Path="/home/icb/manuel.gander/Tome/Data"
# Define time points
t=[3.5, 4.5, 5.25, 5.5]+[6.25+x/4 for x in range(0,10)]+[9.5+x for x in range(0,5)]
ts=[str(a) for a in [3.5, 4.5, 5.25, 5.5, 6.25]] + [str(a) for a in np.arange(6.5, 8.5, 0.25)] + ['8.5a', '8.5b']+ [str(a) for a in np.arange(9.5, 14.5, 1)]
ts=['E'+a for a in ts]

# Utils

## Validation metrics

In [3]:
#################################      Germ-layer metric       ####################################################


def germ_metric(T, cutoff=0, pr=False):

    
    G=map_to_tree(T, cutoff=cutoff)

    #######################################       Score calculation     #########################################
    c=0
    w=0
    for e in G.edges():
        ew=G.edges[e]['edge_weight']
        a0=e[0]
        if a0=='Morula':
            a0=a0
        else:
            a0=a0.split(':')[1]
        a1=e[1].split(':')[1]
        g0=GT[a0]
        #print(a0)
        g1=GT[a1]
        if g0=='other' or g1=='other' or g0==g1:
                c=c+ew
        # The transition from neural crest to (cranial) osteeoblast is the only known differentitaion path that is 
        # known to violates germ lines. Thus we don't punish these connections.

        elif a0=='Neural crest' and a1=='Osteoblast progenitors A':
            #print('here')
            c=c+ew
        elif a0=='Neural crest' and a1=='Osteoblast progenitor B':
            c=c+ew
        else:
            w=w+ew
    r=w/(c+w)*100
    if pr==True:
        print(r)
        print('---------------------------------')
        print(c)
        print(w)
    return(r)

    
#################################     Consistency metric      ####################################################

def cons_metric(T, pr=False):
    C=np.load(f"{Path}/Centromeres.npy", allow_pickle=True)
    C = dict(enumerate(C.flatten(), 1))[1]
    
    error=np.zeros(len(ts)-1)

    for i in range(0,len(ts)-1):
        # Construct child-marignal
        cats=T[ts[i]].obs['cell_state_child']
        marg_c=np.zeros(len(cats), dtype=np.float64)
        for j in range(0,len(cats)):
            if not cats[j] in skip_these:
                marg_c[j]=C[f'{ts[i]}_{cats[j]}'][2]
        marg_c=marg_c/marg_c.sum()

        # Use child marginal and coupling to predict parent maringinal
        marg_p=np.matmul(T[ts[i]].X.T, marg_c)

        # Exclude pull of missing cell frations
        cats=T[ts[i]].var['cell_state_parent']
        for j in range(0,len(cats)):
            if cats[j] in skip_these:
                marg_p[j]=0
        marg_p=marg_p/marg_p.sum()


        # The measured parent marginal
        cats=T[ts[i]].var['cell_state_parent']
        marg_m=np.zeros(len(cats), dtype=np.float64)
        for j in range(0,len(cats)):
            if not cats[j] in skip_these:
                marg_m[j]=C[f'{ts[i]}_{cats[j]}'][2]
        marg_m=marg_m/marg_m.sum()


        # The transportation cost:
        cats=T[ts[i]].var['cell_state_parent']
        centromeres=np.zeros((len(cats),30))
        for j in range(0,len(cats)):
            if not cats[j] in skip_these:
                centromeres[j]=C[f'{ts[i]}_{cats[j]}'][1]

        cost=sklearn.metrics.pairwise_distances(centromeres, centromeres, metric='sqeuclidean')
        cost=cost/cost.mean()

        error[i] = ot.emd2(marg_p, marg_m, cost)
    if pr==True:
        print(error.sum())
    return error.sum()

    
###################################    Literature metric    ##################################################
def lit_metric(T, cutoff=0, pr=False):
    
    G=map_to_tree(T, cutoff=cutoff)
    
    c=0
    w=0
    for g in G.edges:
        if g in Gt.edges:
            c=c+G.edges[g]['edge_weight']
        else:
            w=w+G.edges[g]['edge_weight']
    r=w/(c+w)*100
    if pr==True:
        print(r)
        print('------------------')
        print(c)
        print(w)
    return(r)

## Literature tree

In [4]:
# Calculate a d3-tree given a networkx-tree  
# The function returns a string, which has to be copied into a .json file, as descrived in Overview.txt

def jsontree(G):
    E=[]
    pl=['Morula']

    S1='{\"name\": \"Morula\", \"zindex\": 50'
    S2=[]

    z=0
    k=0
    for j in range(0,10000):
        n=list((G[f'{pl[-1]}']))

        #if pl==[]:
            #print(j)
            #continue
        if n==[]:
            pl=pl[:-1].copy()

            k=k+1

            if k==1:
                S1=S1+S2[-1]
            if k>1.5:
                S1=S1+S2[-1]+']'
            S2=S2[:-1]
        else:
            l=0
            for i in n:
                if l==1:
                    continue
                elif f'({pl[-1]}, {i})' not in E:
                    #print([pl[-1], i])
                    E=E+[f'({pl[-1]}, {i})']
                    if k==0:
                        S1=S1+', \"children\": [{ \"name\": \"' + f'{i}' + '\", \"zindex\": 50 '
                        k=0
                    if k>0.5:
                        S1=S1+', { \"name\": \"' + f'{i}' + '\", \"zindex\": 50 '
                        k=0

                    ew=G.edges[(f'{pl[-1]}', f'{i}')]['edge_weight']

                    fxr=(i.split(':')[1])
                    fx=CTi[f'{fxr}']

                    gt=GT[f'{fxr}']

                    S2=S2+[', \"edge_weight\":'+f'{ew}, '+'\"fx\": \"'+f'{fx}'+'\", \"node_group\": \"' +f'{gt}'+'\"}']
                    pl=pl+[i]
                    l=1
            if l==0:
                if pl==['Morula']:
                    continue
                else:
                    pl=pl[:-1].copy()

                    k=k+1

                    if k==1:
                        S1=S1+S2[-1]
                    if k>1.5:
                        S1=S1+']'+S2[-1]
                    S2=S2[:-1]
    S1=S1+'], "fx": "94", "node_group": "other"}'
    print(S1)


# Transform a map describing the full mouse embryo development into a networkx-tree
def map_to_tree(T, cutoff=0.2):

    # Create graph and add (unconnected) nodes
    G= nx.DiGraph()
    G.add_node('Morula')
    for i in list(T.values()):
        s=list(i.var['cell_state_parent'])
        for j in s:
            G.add_node(j)
    s=list(list(T.values())[-1].obs['cell_state_child'])
    for j in s:
        G.add_node(j)

    # Add edges
    G= nx.DiGraph()
    G.add_edge('Morula', 'E3.5:Inner cell mass', edge_weight=1.00)
    G.add_edge('Morula', 'E3.5:Trophectoderm', edge_weight=1.00)
    for i in list(T.values()):
        s=list(i.var['cell_state_parent'])
        ts=list(i.obs['cell_state_child'])
        for j in range(0,len(s)):
            for k in range(0, len(ts)):
                a=i.X[k,j]
                if a>cutoff:
                    #print(a)
                    if a==1:
                        a=1
                    #else: 
                    #    a=((-np.log10(1-a)/10)-0.5)*2

                    G.add_edge(i.var['cell_state_parent'][j], i.obs['cell_state_child'][k], edge_weight=a)
                    
    return(G)

## Construct curated literature-tree

In [5]:
######## Load the excel-file where all transitions between cell types corresponds to literature    ############## 

Lit=pd.read_csv(f'{Path}/tome_lit.csv')


###################     Check for inconsistencys     ##########################################################
Dict={}
k=0
for i in range(0,len(Lit)):
    ct_l=Lit['Cell type']
    Dict[f'{ct_l[i]}']=i

# Check if all the decendants are also in progenitors
for i in range(0, len(Lit)):
    ct=Lit['Cell type'][i]
    #print(i)
    for j in Lit['known descendants'][i].split(', '):
        if j=='/':
            pass
        else:
            if ct in Lit['known progenitors'][Dict[j]].split(', '):
                pass
            else:
                print(f'{ct} is not in prog of {j}')
                print(Lit['known progenitors'][Dict[j]].split(', '))
                k=k+1
# Check if all the progenitors are also in descendents
for i in range(0, len(Lit)):
    ct=Lit['Cell type'][i]
    #print(i)
    for j in Lit['known progenitors'][i].split(', '):
        #print(j)
        if j=='/':
            pass
        else:
            if ct in Lit['known descendants'][Dict[j]].split(', '):
                pass
            else:
                print(f'{ct} is not in desc of {j}')
                k=k+1
if k==0:
    print('All good!')
    
#######################################       Graph construction     #########################################

# Take the nodes from TOME            
T=np.load(f'{Path}/Tree/Tome_transition_rates.npy', allow_pickle=True)
T = dict(enumerate(T.flatten(), 1))[1]

# Create graph and add (unconnected) nodes
G_TOME=map_to_tree(T, cutoff=0)


# Add the nodes
G= nx.DiGraph()
for n in G_TOME.nodes:
    G.add_node(n)

# Utils for adding the edges
dd={}
for i in range(0,len(Lit)):
    dd[Lit['Cell type'][i]]=i
    
    
def cell_type_divergence(cs):
    if cs=='Paraxial mesoderm A' or cs=='Paraxial mesoderm B' or cs=='Paraxial mesoderm C':
        cs='Paraxial mesoderm'

    elif cs=='Amniochorionic mesoderm A' or cs=='Amniochorionic mesoderm B':
        cs='Amniochorionic mesoderm'

    elif cs=='Osteoblast progenitors A' or cs=='Osteoblast progenitors B':
        cs='Osteoblast progenitors'
    if cs=='Midgut/Hindgut epithelium':
        cs='Midgut/hindgut epithelium'
    return(cs)
        
# Adding the edges
ew=1

G.add_edge('Morula', 'E3.5:Inner cell mass', edge_weight=ew)
G.add_edge('Morula', 'E3.5:Trophectoderm', edge_weight=ew)

for tp0 in ts[:-1]:
    nodes0=[a for a in T[tp0].var['cell_state_parent']]
    nodes1=[a for a in T[tp0].obs['cell_state_child']]
    for i in nodes0:
        for j in nodes1:
            # Get both cell types
            cs0=i.split(':')[1]
            cs1=j.split(':')[1]
            
            # Adjust naming for some cell types
            cs0=cell_type_divergence(cs0)
            cs1=cell_type_divergence(cs1)  
            
            # Check if a trasition between these two cell types is conform with literature
            
            des=Lit['known descendants'][dd[cs0]]
            des=des.split(', ')
            
            if cs1 in des or cs0==cs1:
                G.add_edge(i, j, edge_weight=ew)
                
Gt=G.copy()

All good!


# Utility functions

In [6]:
def add_par(T, ts0, name, obsm=False):
    
    A=T[f'{ts0}']
    
    M=np.zeros((A.X.shape[0], A.X.shape[1]+1), dtype=np.float64)
    M[:,:-1]=A.X

    A1=anndata.AnnData(X=M)
    A1.X=M
    A1.obs=A.obs
    A1.var['cell_state_parent']=[a for a in A.var['cell_state_parent']]+ [f'{name}']
    
    if obsm==True:
        M2=np.zeros(M.shape)
        M2[:,:-1]=A.obsm['Xn']
        M3=np.zeros(M.shape)
        M3[:,:-1]=A.obsm['Xp']
        A1.obsm['Xn']=M2
        A1.obsm['Xp']=M3
    
    T[f'{ts0}']=A1
    
def add_chi(T, ts0, name, obsm=False):
        
    A=T[f'{ts0}']
    
    M=np.zeros((A.X.shape[0]+1, A.X.shape[1]), dtype=np.float64)
    #print(type(M[0,0]))
    M[:-1,:]=A.X
    #print(type(M[0,0]))
    
    D={}
    for i in range(0,len(ts)):
        D[ts[i]]=i
    ts1=ts[D[ts0]+1]
    A1=anndata.AnnData(X=M)
    A1.X=M
    A1.var=A.var
    A1.obs['cell_state_child']=[a for a in A.obs['cell_state_child']]+ [f'{name}']
    
    if obsm==True:
        M2=np.zeros(M.shape)
        M2[:,:-1]=A.obsm['Xn']
        M3=np.zeros(M.shape)
        M3[:,:-1]=A.obsm['Xp']
        A1.obsm['Xn']=M2
        A1.obsm['Xp']=M3
    
    T[f'{ts0}']=A1
    
# The following cell states are skipped w.r.t. consistency score:
#    - E3.5 / E4.5:Trophectorderm', because the E3.5/E4.5 data set doesn't contain any trophectoderm-cells 
#    - E6.75:Parietal endoderm, becuase there are no parietal endoderm cells at time E6.75 (porobaly undersampling)

skip_these=['E3.5:Trophectoderm', 'E4.5:Trophectoderm', 'E6.75:Parietal endoderm']

def calc_wrong_edges(T, cutoff=0.2):
    G=map_to_tree(T, cutoff=0.2)
    for e in G.edges():
        if e not in Gt.edges():
            ew=G.edges[e]['edge_weight']
            ew=np.round(ew*1000)/1000
            print(f'{ew}: {e[0]} to {e[1]}')

In [7]:
# Dictionary of cells types used for tree plot construction and scoring

CT={    "1":  "Neural crest (PNS glia)",
        "2":  "Neural crest (PNS neurons)",
        "3":  "Olfactory sensory neurons",
        "4":  "Neural crest",
        "5":  "Mesencephalon/MHB",
        "6":  "Di/telencephalon",
        "7":  "Retinal neurons",
        "8":  "Retinal pigment cells",
        "9":  "Retinal primordium",
        "10": "Forebrain/midbrain",
        "11": "Noradrenergic neurons",
        "12": "Motor neurons",
        "13": "Intermediate progenitor cells",
        "14": "Inhibitory interneurons",
        "15": "Di/mesencephalon inhibitory neurons",
        "16": "Spinal cord inhibitory neurons",
        "17": "Di/mesencephalon excitatory neurons",
        "18": "Spinal cord excitatory neurons",
        "19": "Neuron progenitor cells",
        "20": "Hindbrain",
        "21": "Roof plate",
        "22": "Anterior floor plate",
        "23": "Posterior floor plate",
        "24": "Spinal cord",
        "25": "Spinal cord (dorsal)",
        "26": "Spinal cord (ventral)",
        "27": "Rostral neuroectoderm",
        "28": "Epiblast",
        "29": "Caudal neuroectoderm",
        "30": "Neuromesodermal progenitors",
        "31": "Caudal lateral epiblast",
        "32": "Otic epithelium",
        "33": "Olfactory epithelium",
        "34": "Placodal area",
        "35": "Branchial arch epithelium",
        "36": "Fusing epithelium",
        "37": "Apical ectodermal ridge",
        "38": "Epidermis",
        "39": "Pre-epidermal keratinocytes",
        "40": "Surface ectoderm",
        "41": "Primitive streak and adjacent ectoderm",
        "42": "Primordial germ cells",
        "43": "Foregut epithelium",
        "44": "Midgut/Hindgut epithelium",
        "45": "Lung epithelium",
        "46": "Pancreatic epithelium",
        "47": "Gut and lung epithelium",
        "48": "Hepatocytes",
        "49": "Gut",
        "50": "Definitive endoderm",
        "51": "Anterior primitive streak",
        "52": "Notochord",
        "53": "Osteoblast progenitors A",
        "54": "Osteoblast progenitors B",
        "55": "Myocytes",
        "56": "Skeletal muscle progenitors",
        "57": "Early chondrocytes",
        "58": "Chondrocyte and osteoblast progenitors",
        "59": "Paraxial mesoderm A",
        "60": "Paraxial mesoderm B",
        "61": "Paraxial mesoderm C",
        "62": "Nascent mesoderm",
        "63": "Intermediate mesoderm",
        "64": "Renal epithelium",
        "65": "Mesenchymal stromal cells",
        "66": "Somatic mesoderm",
        "67": "Connective tissue progenitors",
        "68": "Limb mesenchyme progenitors",
        "69": "Cardiomyocytes",
        "70": "First heart field",
        "71": "Second heart field",
        "72": "Splanchnic mesoderm",
        "73": "Amniochorionic mesoderm A",
        "74": "Amniochorionic mesoderm B",
        "75": "Amniochorionic mesoderm",
        "76": "Extraembryonic mesoderm",
        "77": "Allantois",
        "78": "Mixed mesoderm",
        "79": "Brain endothelium",
        "80": "Liver endothelium",
        "81": "Endothelium",
        "82": "Hematoendothelial progenitors",
        "83": "Blood progenitors",
        "84": "White blood cells",
        "85": "Megakaryocytes",
        "86": "Primitive erythroid cells",
        "87": "Definitive erythroid cells",
        "88": "Embryonic visceral endoderm",
        "89": "Extraembryonic visceral endoderm",
        "90": "Visceral endoderm",
        "91": "Parietal endoderm",
        "92": "Hypoblast",
        "93": "Inner cell mass",
        "94": "Morula",
        "95": "Trophectoderm",
        "96": "Extraembryonic ectoderm"}
CTi=dict((v, k) for k, v in CT.items())


# Assign each cell type a germ layer (where 'other' specifies an undefinied germ layer)
GT=CTi.copy()
for i in GT.keys():
    if float(GT[i])<31.5:
        GT[i]='neuroectoderm'
    elif float(GT[i])>31.5 and float(GT[i])<40.5:
        GT[i]='surface_ectoderm'
    elif float(GT[i])>42.5 and float(GT[i])<50.5:
        GT[i]='endoderm'
    elif float(GT[i])>51.5 and float(GT[i])<87.5:
        GT[i]='mesoderm'
    elif float(GT[i])>87.5:
        GT[i]='exe_germ'
GT['Epiblast']='other'
GT['Primitive streak and adjacent ectoderm']='other'
GT['Primordial germ cells']='other'
GT['Anterior primitive streak']='other'
GT['Inner cell mass']='other'
GT['Morula']='other'

In [1]:
# For each time pair and epsilon a lam1 was determined such that the growth rates result to approximately 2% of
# cell going through apoptosis.
def give_lam1(i, eps):
    if i==0:
        lam1=1.5
    if i==1:
        lam1=3
    if i==2:
        lam1s=[2, 1]
        if eps>0.0002:
            lam1=lam1s[0]
        else:
            lam1=lam1s[1]
    if i==3:
        lam1=0.13
    if i==4:
        lam1s=[1.3, 0.17]
        if eps>0.0002:
            lam1=lam1s[0]
        elif eps==0.0001:
            lam1=0.4
        else:
            lam1=lam1s[1]
    if i==5:
        lam1=0.13
    if i==6:
        lam1s=[0.008, 0.004]
        if eps>0.000002:
            lam1=lam1s[0]
        else:
            lam1=lam1s[1]

    if i==7:
        lam1=0.02
    if i==8:
        lam1s=[0.1, 0.08, 0.03]
        if eps>0.0002:
            lam1=lam1s[0]
        elif eps==0.0001:
            lam1=lam1s[1]
        else:
            lam1=lam1s[2]

    if i==9:
        lam1s=[0.04, 0.02]
        if eps>0.00002:
            lam1=lam1s[0]
        else:
            lam1=lam1s[1]

    if i==10:
        lam1s=[0.6, 0.1, 0.03, 0.02]
        if eps>0.0002:
            lam1=lam1s[0]
        elif eps>0.00002:
            lam1=lam1s[1]
        elif eps==0.00001:
            lam1=lam1s[2]
        else:
            lam1=lam1s[3]
    if i==11:
        lam1s=[3, 0.5, 0.08, 0.05]
        if eps>0.0002:
            lam1=lam1s[0]
        elif eps>0.00002:
            lam1=lam1s[1]
        elif eps==0.00001:
            lam1=lam1s[2]
        else:
            lam1=lam1s[3]
    if i==12:
        lam1s=[0.3, 0.15, 0.04]
        if eps>0.0002:
            lam1=lam1s[0]
        elif eps==0.0001:
            lam1=lam1s[1]
        else:
            lam1=lam1s[2]

    if i==13:
        lam1=0.05
    
    return(lam1)

# Validation for single maps

In [31]:
def run_val(save_key, cutoff=0):
    T=np.load(f'{Path}/Tree/Tome_transition_rates.npy', allow_pickle=True)
    T = dict(enumerate(T.flatten(), 1))[1]

    a=np.zeros(3)
    b=np.zeros(3)
    a[0]=germ_metric(T, cutoff=cutoff)
    a[1]=lit_metric(T, cutoff=cutoff)
    a[2]=cons_metric(T)

    ts0=ts[i]
    #print(ts0)
    V={}
    D=np.load(f'{Path}/Maps/{ts0}_{save_key}_sm.npy', allow_pickle=True)
    D = dict(enumerate(D.flatten(), 1))[1]
    for k in D.keys():
        T=np.load(f'{Path}/Tree/Tome_transition_rates.npy', allow_pickle=True)
        T = dict(enumerate(T.flatten(), 1))[1]
        A=D
        T[ts0]=A[ts0][0]

        # Add missing cell types
        if ts0=='E3.5':
            add_par(T, ts0, 'E3.5:Trophectoderm')
            add_chi(T, ts0, 'E4.5:Trophectoderm')
            T['E3.5'][-1,-1]=1
        if ts0=='E4.5':
            add_par(T, ts0, 'E4.5:Trophectoderm')
            add_chi(T, ts0, 'E5.25:Extraembryonic ectoderm')
            T['E4.5'][-1,-1]=1
        if ts0=='E6.5':
            add_par(T, ts0, 'E6.5:Parietal endoderm')
            add_chi(T, ts0, 'E6.75:Parietal endoderm')
            T['E6.5'][-1,-1]=1
        if ts0=='E6.75':
            add_par(T, ts0, 'E6.75:Parietal endoderm')
            add_chi(T, ts0, 'E7:Parietal endoderm')
            T['E6.75'][-1,-1]=1
        if ts0=='E8.5a':
            add_par(T, ts0, 'E8.5a:Primitive erythroid cells')
            add_chi(T, ts0, 'E8.5a:Primitive erythroid cells')
            T['E8.5a'][-1,-1]=1

        b[0]=germ_metric(T, cutoff=cutoff)
        b[1]=lit_metric(T, cutoff=cutoff)
        b[2]=cons_metric(T)

        V[k]=((b-a)/a*100, A[ts0][1], A[ts0][2], A[ts0][3], A[ts0][4], cutoff)
    return(V[ts0][0], D[k][5]*100)

In [169]:
import warnings
warnings.filterwarnings("ignore")

lam2=50
for i in range(12,14):
    ts0=ts[i]
    print('--------------------------------------------------------------------------------------------')
    print(i)
    fraction=1
    if i<3:
        epss=[0.1, 0.01, 0.001, 0.0001,  0.00001,  0.000001, 0.0000001, 0.00000001]
    else:
        epss=[0.01, 0.001, 0.0001, 0.00001, 0.000001, 0.0000001, 0.00000001]
    for eps in epss:
        
        lam1=give_lam1(i, eps)

        tau1=lam1/(lam1+eps)
        tau2=lam2/(lam2+eps)

        save_key=f'{eps}_{tau1}_{tau2}'
        r=run_val(save_key)


        print(f'{eps}_{lam1}:{r}')

--------------------------------------------------------------------------------------------
12
0.01_0.3:(array([ 2.00835669,  2.37933461, -0.51781275]), 1.9813127470265757)
0.001_0.3:(array([ 1.81100583,  1.67168054, -0.51922166]), 2.0457850893796077)
0.0001_0.15:(array([ 0.63752962,  1.22904293, -0.41497223]), 2.1887284742033537)
1e-05_0.04:(array([-0.77348303,  0.77907173, -0.31136868]), 3.487302329282117)
1e-06_0.04:(array([-2.0530512 ,  0.56815687, -0.30940778]), 2.343337320825647)
1e-07_0.04:(array([-1.87512972,  0.62429882, -0.30715872]), 2.2717449202473263)
1e-08_0.04:(array([-1.88633058,  0.56649174, -0.30678485]), 2.266233143316569)
--------------------------------------------------------------------------------------------
13
0.01_0.05:(array([ 2.34217248,  1.89674798, -0.00339887]), 9.417258737506131)
0.001_0.05:(array([ 2.26528687,  1.52482213, -0.00846762]), 9.513932337768303)


FileNotFoundError: [Errno 2] No such file or directory: '/home/icb/manuel.gander/Tome/Data/Maps/E8.5a_0.0001_0.998003992015968_0.999998000004_sm.npy'